In [1]:
import collections #to sort list by occurrences
from collections import Counter

In [2]:
#get the clean text from the other notebook
%run ./p_03_prepare_text_paper.ipynb
# list_wordcount_drop. list with the numbers that we might drop
# dict_clean_doc_x. has the doc_clean for each x from the list_wordcount_drop
# dict_doc_list_paras_x. has the doc_list_paras for each x. 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### show the top n words when short docs are dropped

In [3]:
#show for each x in list_wordcount_drop the 25 topwords when all docs <=x are dropped
var_top_n = 50
df_top_words_x = pd.DataFrame()
dict_list_all_docs_together = {}
for x in list_wordcount_drop:
    dict_list_all_docs_together['>'+str(x)] = []
    for doc in dict_clean_doc_x['>'+str(x)]:
        dict_list_all_docs_together['>'+str(x)].extend(doc)
    list_sorted_top = Counter(dict_list_all_docs_together['>'+str(x)]).most_common()[0:var_top_n]
    df_top_words_x['>'+str(x)] = list_sorted_top
        

In [4]:
df_top_words_x.to_csv('.//results//LDA_set_parameters//word_drop_lda//qa_top_'+str(var_top_n)+'_words_per_x_'+timestr+'.csv',
    sep=';', decimal=',')

## calculate the relative changes for words

In [6]:
#first, calculate the average change.
#how many words in total are in the corpus for each x -->Average change
dict_changes = {} #the changes in number of total words 
dict_occur = {} #the occurrances of each word
for x in range(0, len(list_wordcount_drop)):
    x_0 = list_wordcount_drop[x]
    dict_occur[str(x_0)] = Counter(dict_list_all_docs_together['>'+str(x_0)])
    if x != len(list_wordcount_drop)-1:
        x_n = list_wordcount_drop[x+1]
        var_total_words = len(dict_list_all_docs_together['>'+str(x_0)])
        var_total_words_n = len(dict_list_all_docs_together['>'+str(x_n)])
        var_change = var_total_words/var_total_words_n
        dict_changes[str(x_0)+'_'+str(x_n)] = var_change

In [7]:
#how does each word deviate from the average change? 
#only for words that occur at least 10 times

dict_word_changes_dict = {}
dict_word_changes_df = {}
for x in range(0, len(list_wordcount_drop)):
    dict_aa = {}
    x_0 = list_wordcount_drop[x]
    if x != len(list_wordcount_drop)-1:
        x_n = list_wordcount_drop[x+1]
        for word in dict_occur[str(x_0)]:
            var_occur = dict_occur[str(x_0)][word]
            if var_occur > 50: #word occurs at least X times
                if word in dict_occur[str(x_n)]: #word occurs in both corpi
                    var_occur_n = dict_occur[str(x_n)][word]
                    if var_occur != var_occur_n:
                        var_occur_change = var_occur/var_occur_n
                        var_change_change = var_occur_change - dict_changes[str(x_0)+'_'+str(x_n)]
                        dict_aa[word] = var_change_change
        
        dict_word_changes_dict[str(x_0)+'_'+str(x_n)] = dict_aa
        dict_word_changes_df[str(x_0)+'_'+str(x_n)] = pd.DataFrame.from_dict(dict_aa, orient='index')
        if dict_aa != {}: #can be empty, if the corpi are too similar
            dict_word_changes_df[str(x_0)+'_'+str(x_n)].columns = [str(x_0)+'_'+str(x_n)]
            dict_word_changes_df[str(x_0)+'_'+str(x_n)].sort_values(str(x_0)+'_'+str(x_n), ascending=False, inplace=True)
        else:
            dict_word_changes_df[str(x_0)+'_'+str(x_n)][str(x_0)+'_'+str(x_n)] = []

In [8]:
#A POSITIVE VALUE IMPLIES THAT THE RESPECTIVE WORD DISAPPEARS
#A NEGATIVE VALUE IMPLIES, THAT THE WORD APPEARS (ALMOST) AS OFTEN

In [9]:
#do a dataframe, showing the top 50 words that change between corpi
df_words_disappear = pd.DataFrame(index=list(range(0,50)))
for x in range(0, len(list_wordcount_drop)):
    x_0 = list_wordcount_drop[x]
    if x != len(list_wordcount_drop)-1:
        x_n = list_wordcount_drop[x+1]
        df_to_add = dict_word_changes_df[str(x_0)+'_'+str(x_n)].reset_index().loc[0:50,:]
        df_to_add.columns = [str(x_0)+'_'+str(x_n)+'_words', str(x_0)+'_'+str(x_n)+'_changes']
        df_words_disappear = df_words_disappear.join(df_to_add)

In [10]:
df_words_disappear.to_csv('.//results//LDA_set_parameters//word_drop_lda//qa_occurrence_word_changes_'+timestr+'.csv',
    sep=';', decimal=',')